# Simon's Algorithm

Given a blackbox function $f$ we seek to determine if it is one-to-one or two-to-one. 

> **One-to-One**: $f(1) \rightarrow 1$, $f(2) \rightarrow 2$, $f(3) \rightarrow 3$, $f(4) \rightarrow 4$  
> **Two-to-One**: $f(1) \rightarrow 1$, $f(2) \rightarrow 2$, $f(3) \rightarrow 1$, $f(4) \rightarrow 2$

A two-to-one mapping is defined by a hidden bitstring $b$, where

$$ \text{Given}\ x_1, x_2\ \text{if}\ f(x_1) = f(x_2) \implies x_1 \oplus x_2 = b$$

which therefore reduces the problem to finding $b$ and determining if $b=00...$ which would imply a one-to-one mapping.

## Classical Solution